<a href="https://colab.research.google.com/github/Gh5al/CDMO/blob/main/SMT/SMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We tried 3 methods:
- successor approach with position variable
- boolean assignment variable + postion variable,
- int assign variable (assign a courier to each item) + position variable .<br>
The best approach is to use successor variable + position variable.<br>
For all the approaches it's fundamentel to use the lower bound constraint as it helps to reduce the search space significantly and provide a solution to instances.

In [2]:
!pip install z3-solver
from z3 import *
import numpy as np
import time
from multiprocessing import Process, Queue


In [3]:
def read_data(filename):
  dist = []
  with open(filename,'r') as f:
    m = int(f.readline().strip())
    n = int(f.readline().strip())
    capacity = [int(s) for s in f.readline().strip().split()]
    size = [int(s) for s in f.readline().strip().split()]
    for i in range(n+1):
      dist.append([int(s) for s in f.readline().strip().split()])
  return m,n,capacity,size,dist

In [4]:
def extract_routes(model,m,n,d_var,succ=False):
  #extract route
  couriers = list(range(m))
  items = list(range(n))
  locs = list(range(n+1))
  if succ:
    iter = locs
  else: iter = items
  routes = []
  for k in couriers:
      #position = model.evaluate(pos[k][i])
    routes.append([model.evaluate(d_var[k][i]).as_long() for i in iter])
  return routes

def extract_sol(routes,n,succ=False):
  sol = []
  items = list(range(n))
  #extract solution for successor approach
  if succ:
    for x in routes:
      sub_sol = []
      prev = x[n]
      while(prev != n):
        for i in items:
          if i == prev:
            sub_sol.append(i+1)
            prev = x[i]
            break
        sol.append(sub_sol)
  else:
    #for routes from position variable
    for x in routes:
      sub_sol = []
      #count the number of items delivered by a courier
      count = sum([1 for p in x if p>0])
      prev = 1
      while(prev != count + 1):
        for i in items:
          if x[i] == prev:
            sub_sol.append(i+1)
            prev = prev+1
        sol.append(sub_sol)
  return sol

In [47]:
def solve(q,m,n,solver,d_var,obj,succ=False):
    info = {'obj':0, 'routes':[]}
    # Try to get intermediate results
    while solver.check() == sat:
      model = solver.model()
      sub_obj = model.evaluate(obj)
      print(f"current_obj_value: {sub_obj}\n")
      routes = extract_routes(model,m,n,d_var,succ=succ)
      info.update({'obj':sub_obj.as_long(), 'routes':routes})
      q.put(info)
      #try to improve the current obj
      solver.add(obj< sub_obj)

def run_with_timeout(m,n,solver,d_var,obj,timeout,succ=False):
    q = Queue()
    p = Process(target=solve, args=(q,m,n,solver,d_var,obj,succ))
    p.start()
    p.join(timeout=timeout)

    if p.is_alive():
        p.terminate()
        print("Timeout reached.")
    else:
        print("Finished before timeout.")

    # Get last model found before termination
    best_obj = 0
    while not q.empty():
        #take from the queue the dictionary containing the information about obj and routes
        info = q.get()

    if info['obj']>0:
       # print("Intermediate solution found:")
        print(f"Objective:{info['obj']}")
    else:
        print("No solution found in time.")
    return info



## SUCCESSOR MODEL + POSITION VARIABLE

In [31]:

def run_successor_model(filename):
  #extract data from instance file
  m,n,capacity,size,dist = read_data(filename)
  print(f"num_couriers:{m}, num_items: {n}")
  items = list(range(n))
  locs = list(range(n + 1))
  couriers = list(range(m))
  #SUCCESSOR MODEL with position variable
  solver = Solver()
  start_time = time.time()

  # --- VARIABLES ---

  # S[k][i] = j if courier k delivers item i and then delivers item j
  S = [[Int(f"S_{k}_{i}") for i in locs] for k in couriers]

  # pos[k][i]: order index of node i visited by courier k
  pos = [[Int(f"pos_{k}_{i}") for i in items] for k in couriers]

  # distance[k]: total distance of courier k
  distance = [Int(f"distance_{k}") for k in couriers]

  #objective function: minimize the maximum distance travelled by any courier
  objective = Int("objective")

  #----- CONSTRAINTS -----

  #bound the possible values the can be assigned to S variable
  for k in couriers:
    for i in locs:
      solver.add(And(S[k][i]>=0, S[k][i]<=n))

  #each item should be delivered by only one courier
  for i in items:
    count = Sum([If(S[k][i]!= i,1,0) for k in couriers])
    solver.add(count==1)

  #each courier should leave the depot
  for k in couriers:
    solver.add(S[k][n] != n)


  # ---- IMPLIED CONSTRAINTS -----

  #all courier leaves from the depot to a different place
  solver.add(Distinct([S[k][n] for k in couriers]))

  #each courier should return to depot only once ()
  for k in couriers:
    solver.add(Sum([If(S[k][i] == n,1,0) for i in range(n)])==1)

  #for each courier the successor's values should be all different
  for k in couriers:
    #a solution can be found without this constraint, but it's very slow
    solver.add(Distinct([S[k][i] for i in locs]))

  # ---- IMPLIED CONSTRAINTS -----

  #capacity constraint
  for k in couriers:
    solver.add(Sum([If(S[k][i]!= i,size[i],0) for i in items]) <= capacity[k])

  #all values in position[k] array should be different, each node appears in the route only once
  for k in couriers:
    solver.add(Distinct([pos[k][i] for i in items]))

  #prevent unconnected routes constraint
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(S[k][i]!= i,1,0) for i in items])
    for i in items:
      #if an item is assigned to a courier k, then the position should have a value between 1 and num_assigned items to the courier k
      solver.add(Implies(S[k][i]!= i, And(pos[k][i] >= 1, pos[k][i] <= num_assigned)))
      solver.add(Implies(S[k][i]==i,pos[k][i]<0))

  #Distance calculation
  for k in couriers:
    #requires two loops because S is a symbolic variable and can't be used as index in the distance matrix
      solver.add(distance[k] == Sum([Sum([If(S[k][i] == j, dist[i][j], 0) for j in locs]) for i in locs]))

  #constraint the obj to be the biggest distance travelled by any courier
  for k in couriers:
      solver.add(distance[k] <= obj)

  #compute objective lowerbound
  lower_bound = max([dist[n][i] + dist[i][n] for i in items])
  print(f"lower_bound: {lower_bound}")
  solver.add(objective>=lower_bound)
  #the max distance for a courier could be to deliver n-m+1 items with longest distance between depot and delivery point
  #sorted_distances=sorted([dist[n][i]+dist[i][n] for i in range(n)],reverse=True)
  #print(sorted_distances)
  #upper_bound = sum(sorted_distances[:n-m+1])
  #print(f"upper_bound: {upper_bound}")
  #solver.add(max_distance<=upper_bound)

  encoding_time = time.time() - start_time
  print(f"encoding_time: {encoding_time:3.2f} secs\n")
  # Set timeout to 5 minutes(300 secs)(include also the encoding time as for large instances it can be remarkable)
  timeout = 300 - encoding_time
  solver.set("timeout", int(timeout*1000))
  search_start_time = time.time()

  #run the searching using the Process and Queue library to abort execution after reaching timeout
  #in this approach the routes are extracted from the successor variable
  info = run_with_timeout(m,n,solver,S,objective,timeout,succ=True)

  #extract the items assigned to each courier
  sol = extract_sol(info['routes'],n,succ=True)

  searching_time = time.time() - search_start_time
  final_time = searching_time + encoding_time
  print(f"Finished in: {final_time:3.2f} secs\n")
  return final_time, info['obj'], sol

## BOOLEAN VARIABLE + POSITION VARIABLE

In [40]:
#bool var + pos var
def run_boolean_model(filename):
  #extract data from instance file
  m,n,capacity,size,dist = read_data(filename)
  print(f"num_couriers:{m}, num_items: {n}")
  items = list(range(n))
  locs = list(range(n + 1))
  couriers = list(range(m))
  solver = Solver()
  start_time = time.time()

  # --- VARIABLES ---

  # A[k][i] = 1 if courier k delivers item i
  A = [[Bool(f"A_{k}_{i}") for i in items] for k in couriers]

  # pos[k][i]: order index of node i in the route of courier k
  pos = [[Int(f"pos_{k}_{i}") for i in items] for k in couriers]

  # distance[k]: total distance of courier k
  distance = [Int(f"distance_{k}") for k in couriers]

  #objective function: minimize the maximum distance travelled by any courier
  objective = Int("objective")

  # --- CONSTRAINTS ---

  #each item should be delivered by one courier
  for i in range(n):
      solver.add(Sum([If(A[k][i],1,0) for k in couriers]) == 1)

  #capacity constraints
  for k in couriers:
    solver.add(Sum([If(A[k][i],size[i],0) for i in items]) <= capacity[k])

  #for each courier all the position assigned should be different
  for k in couriers:
    solver.add(Distinct([pos[k][i] for i in items]))

  #prevent unconnected routes between delivered items by a courier k
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(A[k][i],1,0) for i in items])
    for i in range(n):
      #if an item is assigned to a courier k, then the position should have a value between 1 and num_assigned items to the courier k
      solver.add(Implies(A[k][i],And(pos[k][i]>=1,pos[k][i]<=num_assigned)))
      solver.add(Implies(Not(A[k][i]),pos[k][i]<0))

  #Distance calculation
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(A[k][i],1,0) for i in items])

    #distance from depot to the first delivery
    depot_to_first= Sum([If(And(A[k][i], pos[k][i] == 1),dist[n][i],0) for i in items])

    #create 2 for with i,j, if an item i and j are delivered, and their position only differs of 1, take all the pairs and then sum all the distances
    betweem_distance = Sum([Sum([ If(And(A[k][i],A[k][j],pos[k][j] == pos[k][i]+1),dist[i][j],0) for j in items if j!=i]) for i in items])

    #distance from last delivery to depot
    last_to_depot = Sum([If(And(A[k][i], pos[k][i] == num_assigned),dist[i][n],0) for i in items])

    #total distance
    solver.add(distance[k] == depot_to_first + betweem_distance + last_to_depot)

  #constraint the obj to be the biggest distance travelled by any courier
  for k in couriers:
      solver.add(distance[k] <= obj)

  #compute objective lowerbound
  lower_bound = max([dist[n][i] + dist[i][n] for i in items])
  solver.add(objective>=lower_bound)
  print(f"lower_bound: {lower_bound}")
  #upper_bound = sum([dist[n][i] for i in range(n)]) + sum([dist[i][n] for i in range(n)])
  #print(upper_bound)
  #sorted_distances=sorted([dist[n][i]+dist[i][n] for i in range(n)],reverse=True)
  #upper_bound = sum(sorted_distances[:n-m+1])
  #print(upper_bound)
  #solver.add(max_distance<=upper_bound)

  encoding_time = time.time() - start_time
  print(f"encoding_time: {encoding_time:3.2f} secs\n")
  # Set timeout to 5 minutes(300 secs)(include also the encoding time as for large instances it can be remarkable)
  timeout = 300 - encoding_time
  solver.set("timeout", int(timeout*1000))
  search_start_time = time.time()

  #run the searching using the Process and Queue library to abort execution after reaching timeout
  #in this approach the routes are extracted from position variable
  info = run_with_timeout(m,n,solver, pos, objective, timeout)

  #extract the items assigned to each courier
  sol = extract_sol(info['routes'],n)

  searching_time = time.time() - search_start_time
  final_time = searching_time + encoding_time
  print(f"Finished in: {final_time:3.2f} secs\n")
  return final_time, info['obj'], sol


# MODEL 3: ASSIGN(INT) VARIABLE + POSITION VARIABLE

In [57]:
def run_int_assign_model(filename):
  #extract data from instance file
  m,n,capacity,size,dist = read_data(filename)
  print(f"num_couriers:{m}, num_items: {n}")
  items = list(range(n))
  locs = list(range(n + 1))
  couriers = list(range(m))
  solver = Solver()
  start_time = time.time()

  # --- VARIABLES ---

  # assign[i] = k if courier k delivers item i
  assign = [Int(f"assign_{i}") for i in items]

  # pos[k][i]: order index of node i in the route of courier k
  pos = [[Int(f"pos_{k}_{i}") for i in items] for k in couriers]

  # distance[k]: total distance of courier k
  distance = [Int(f"distance_{k}") for k in couriers]

  #objective function: minimize the maximum distance travelled by any courier
  objective = Int("objective")

  # --- CONSTRAINTS ---
  #each item should be delivered by a courier, bound the assign variabel
  for i in items:
    solver.add(And(assign[i] >= 0, assign[i] <= m-1))

  #capacity constraint
  for k in couriers:
    solver.add(Sum([If(assign[i]==k,size[i],0) for i in items]) <= capacity[k])

  #for each courier all the position assigned should be different
  for k in couriers:
    solver.add(Distinct([pos[k][i] for i in items]))

  #prevent unconnected routes between delivered items by a courier k
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(assign[i]==k,1,0) for i in items])
    for i in items:
      #if an item is assigned to a courier k, then the position should have a value between 1 and num_assigned items to the courier k, other < 0
      solver.add(Implies(assign[i]==k, And(pos[k][i]>=1, pos[k][i]<=num_assigned)))
      solver.add(Implies(Not(assign[i]==k),pos[k][i]<0))

  #Distance calculation
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(assign[i]==k,1,0) for i in items])
    #distance from depot to the first delivery
    depot_to_first= Sum([If(And(assign[i]==k,pos[k][i] == 1),dist[n][i],0) for i in items])
    #create 2 for with i,j, if an item i and j is delivered, and their position only differs of 1, take all the pairs and then sum all the distances
    betweem_distance = Sum(
        [Sum([If(And(assign[i]==k,assign[j]==k,pos[k][j] == pos[k][i]+1),dist[i][j],0) for j in items if j!=i]) for i in items])
    #distance from last delivery to depot
    last_to_depot = Sum([If(And(assign[i]==k,pos[k][i] == num_assigned) ,dist[i][n],0) for i in items])
    #total distance
    solver.add(distance[k] == depot_to_first + betweem_distance + last_to_depot)

  #constraint the obj to be the biggest distance travelled by any courier
  for k in couriers:
      solver.add(distance[k] <= objective)

  #compute objective lowerbound
  lower_bound = max([dist[n][i] + dist[i][n] for i in items])
  solver.add(objective>=lower_bound)
  print(f"lower_bound: {lower_bound}")
  #upper_bound = sum([dist[n][i] for i in range(n)]) + sum([dist[i][n] for i in items])
  #print(upper_bound)
  #sorted_distances=sorted([dist[n][i]+dist[i][n] for i in items],reverse=True)
  #upper_bound = sum(sorted_distances[:n-m+1])
  #print(upper_bound)
  #solver.add(max_distance<=upper_bound)

  encoding_time = time.time() - start_time
  print(f"encoding_time: {encoding_time:3.2f} secs\n")
  # Set timeout to 5 minutes(300 secs)(include also the encoding time as for large instances it can be remarkable)
  timeout = 300 - encoding_time
  solver.set("timeout", int(timeout*1000))
  search_start_time = time.time()

  #run the searching using the Process and Queue library to abort execution after reaching timeout
  #in this approach the routes are extracted from position variable
  info = run_with_timeout(m,n,solver, pos, objective, timeout)

  #extract the items assigned to each courier
  sol = extract_sol(info['routes'],n)

  searching_time = time.time() - search_start_time
  final_time = searching_time + encoding_time
  print(f"Finished in: {final_time:3.2f} secs\n")
  return final_time, info['obj'], sol

# RUN MODEL

In [60]:
import math
filename = "./inst06.dat"
#t, obj, sol = run_successor_model(filename)
#t, obj, sol = run_boolean_model(filename)
t, obj, sol = run_int_assign_model(filename)
print(math.floor(t), obj, sol)

num_couriers:6, num_items: 8
lower_bound: 322
encoding_time: 0.59 secs

current_obj_value: 598

current_obj_value: 396

current_obj_value: 395

current_obj_value: 394

current_obj_value: 393

current_obj_value: 392

current_obj_value: 391

current_obj_value: 390

current_obj_value: 389

current_obj_value: 388

current_obj_value: 387

current_obj_value: 386

current_obj_value: 385

current_obj_value: 384

current_obj_value: 383

current_obj_value: 382

current_obj_value: 381

current_obj_value: 380

current_obj_value: 379

current_obj_value: 378

current_obj_value: 377

current_obj_value: 376

current_obj_value: 375

current_obj_value: 374

current_obj_value: 373

current_obj_value: 372

current_obj_value: 371

current_obj_value: 370

current_obj_value: 369

current_obj_value: 368

current_obj_value: 367

current_obj_value: 366

current_obj_value: 365

current_obj_value: 364

current_obj_value: 363

current_obj_value: 362

current_obj_value: 361

current_obj_value: 360

current_obj_valu

In [23]:
import json
import os
import math
import re

def sol_to_json(t,obj,sol,filename,approach_name):
  method = "SMT"

  instance_id = re.search(r'\d+', filename).group()

  if t<300:
    optimal = "true"
    time = math.floor(t)
  else:
    optimal = "false"
    time = 300

  # Create JSON structure
  res = {
      approach_name: {
          "time": time,
          "optimal": optimal,
          "obj": int(obj),
          "sol": sol
      }
  }
  # Create directory
  out_dir = f"res/{method}"
  os.makedirs(out_dir,exist_ok=True)

  # Save solution to file
  with open(f"{out_dir}/{instance_id}.json", "w") as f:
      json.dump(res, f)


In [24]:
approach_name = "boolean"
sol_to_json(t, obj, sol, filename,approach_name)